In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


In [2]:
# Store Part I results into DataFrame
# Load the csv exported in Part I (WeatherPyto a DataFrame)
cities = pd.read_csv("city_data.csv", encoding="utf-8")
cities.head()



,Unnamed: 0,City,Country,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed
0,0,Naze,JP,28.37,129.48,66.470,52,75,3.10
1,1,Ereymentau,KZ,51.62,73.10,15.116,81,61,8.27
2,2,New Norfolk,AU,-42.78,147.06,70.268,39,40,7.20
3,3,Faanui,PF,-16.48,-151.75,83.084,76,100,8.09
4,4,Albany,US,42.60,-73.97,26.870,45,75,3.60


In [3]:
### Humidity Heatmap
# * Configure gmaps.
# * Use the Lat and Lng as locations and Humidity as the weight.
# * Add Heatmap layer to map.
humidity = cities["Humidity"].astype(float)
maxhumidity = humidity.max()
locations = cities[["Latitude", "Longitude"]]

In [4]:
gmaps.configure(api_key=g_key)
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,dissipating=False, max_intensity=maxhumidity,point_radius=3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [5]:
# Create new DataFrame fitting weather criteria
# Narrow down the cities to fit weather conditions.
# Drop any rows will null values.
narrowed_city_df = cities.loc[(cities["Temperature"] > 70) & (cities["Temperature"] < 80) & (cities["Cloudiness"] == 0), :]
narrowed_city_df = narrowed_city_df.dropna(how='any')
narrowed_city_df.reset_index(inplace=True)
del narrowed_city_df['index']
narrowed_city_df.head()

,Unnamed: 0,City,Country,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed
0,43,Guanica,PR,17.97,-66.91,77.270,91,0,1.60
1,49,Cidreira,BR,-30.18,-50.21,75.578,78,0,5.89
2,86,Villa Carlos Paz,AR,-31.42,-64.50,71.276,93,0,1.50
3,164,Savelugu,GH,9.62,-0.83,79.646,13,0,3.10
4,173,Siguiri,GN,11.67,-9.50,71.420,15,0,1.31


In [6]:
# Hotel Map
# Store into variable named hotel_df.
# Add a "Hotel Name" column to the DataFrame.
# Set parameters to search for hotels with 5000 meters.
# Hit the Google Places API for each city's coordinates.
# Store the first Hotel result into the DataFrame.
# Plot markers on top of the heatmap.
hotellist = []

for i in range(len(narrowed_city_df)):
    lat = narrowed_city_df.loc[i]['Latitude']
    lng = narrowed_city_df.loc[i]['Longitude']

    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "key": g_key
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get(base_url, params=params)
    jsn = requested.json()
    try:
        hotellist.append(jsn['results'][0]['name'])
    except:
        hotellist.append("")
narrowed_city_df["Hotel Name"] = hotellist
narrowed_city_df = narrowed_city_df.dropna(how='any')
narrowed_city_df.head()

,Unnamed: 0,City,Country,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,Hotel Name
0,43,Guanica,PR,17.97,-66.91,77.270,91,0,1.60,Guanica
1,49,Cidreira,BR,-30.18,-50.21,75.578,78,0,5.89,CIDREIRA
2,86,Villa Carlos Paz,AR,-31.42,-64.50,71.276,93,0,1.50,Villa Carlos Paz
3,164,Savelugu,GH,9.62,-0.83,79.646,13,0,3.10,Savelugu
4,173,Siguiri,GN,11.67,-9.50,71.420,15,0,1.31,Diambaya


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = narrowed_city_df[["Latitude", "Longitude"]]

In [8]:

fig = gmaps.figure()
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))